In [1]:
import txjupyter
reactor = txjupyter.start('tcp:port=8080')

In [4]:
from klein import route
@route('/')
def home(request):
    return 'Hello, world!'

In [5]:
@route('/greet/<user>')
def froop(request, user):
    return 'Hello {}!'.format(user)

In [14]:
import json
from twisted.internet import defer
import treq

@route('/ip')
@defer.inlineCallbacks
def getip(request):
    url = 'http://httpbin.org/ip'
    response = yield treq.get(url)
    content = yield response.content()
    value = json.loads(content)
    ip = value['origin']
    defer.returnValue('From {}'.format(ip))

In [15]:
@route('/slow')
@defer.inlineCallbacks
def slow(request):
    url = 'http://httpbin.org/delay/10'
    response = yield treq.get(url)
    content = yield response.content()
    value = json.loads(content)
    first_ip = value['origin']
    response = yield treq.get(url)
    content = yield response.content()
    value = json.loads(content)
    second_ip = value['origin']    
    defer.returnValue('These are the same:{}=={}'.format(first_ip, second_ip))

In [17]:
@route('/less-slow')
def less_slow(request):
    url = 'http://httpbin.org/delay/10'
    d = treq.get(url)
    d.addCallback(lambda response: response.content())
    d.addCallback(json.loads)
    d.addCallback(lambda dct: dct['origin'])
    d2 = treq.get(url)
    d2.addCallback(lambda response: response.content())
    d2.addCallback(json.loads)
    d2.addCallback(lambda dct: dct['origin'])
    both = defer.gatherResults([d, d2])
    def do_format(ips):
        first_ip, second_ip = ips
        return 'These are the same:{}=={}'.format(first_ip, second_ip)
    both.addCallback(do_format)
    return both